In [105]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet,TheilSenRegressor,HuberRegressor,RANSACRegressor
from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score,median_absolute_error
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split, RandomizedSearchCV
from sklearn.preprocessing import PolynomialFeatures
import warnings
import re
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')


In [106]:
data = pd.read_csv("ensemble_methods_data.csv")
data.head(3)

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,x12,x13,x14,x15,x16,x17,x18,x19,x20,y
0,0.895355,-0.804316,-0.241497,-0.491154,0.454741,-0.061764,0.253165,-0.256180,-0.746578,-0.205859,...,-0.516430,-0.033613,0.479442,-0.196467,0.629530,-0.631578,-0.284875,-1.548618,-0.176774,-40.982233
1,0.885577,0.238499,-1.078315,0.917768,0.433750,0.358918,-1.364676,-0.656835,-1.116451,-0.440902,...,0.215669,-0.276335,-1.039793,-0.352546,0.139821,1.781685,-0.310396,0.459204,0.987862,155.535020
2,1.112047,-0.089336,0.268167,0.193799,0.629558,0.273311,-1.506922,-0.215568,0.169945,-2.021002,...,-0.885326,-0.604719,-1.204330,-0.772637,-1.947340,-0.551516,1.027070,-1.188385,-0.160905,-83.416703


In [107]:
data.corr()

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,x12,x13,x14,x15,x16,x17,x18,x19,x20,y
x1,1.000000,0.061659,-0.021303,-0.035715,-0.035586,-0.024176,-0.006068,-0.003470,0.047830,-0.076999,...,-0.026976,0.014025,0.012794,0.035006,0.041788,-0.016737,-0.001066,0.027426,-0.004074,0.511054
x2,0.061659,1.000000,0.041680,0.009279,0.006265,-0.003558,0.000466,0.005991,-0.024235,0.038137,...,0.048036,-0.000205,-0.000848,-0.013075,0.004182,-0.036721,0.009377,0.030178,-0.024946,0.576082
x3,-0.021303,0.041680,1.000000,0.019490,0.019917,-0.008439,-0.025133,-0.083497,0.028410,-0.002219,...,-0.018018,-0.008773,0.006525,0.009678,0.014579,-0.009527,0.006718,0.007453,0.008082,-0.002010
x4,-0.035715,0.009279,0.019490,1.000000,0.040645,0.015536,0.003599,0.002659,0.022786,-0.011188,...,-0.017461,-0.083990,-0.028994,-0.019366,0.045903,0.016852,0.026554,-0.033323,0.020411,-0.000311
x5,-0.035586,0.006265,0.019917,0.040645,1.000000,0.036469,-0.003334,-0.011862,-0.061521,-0.011494,...,-0.025125,-0.010997,0.022061,-0.010880,-0.047100,0.029248,0.011127,0.000252,-0.008161,-0.001464
x6,-0.024176,-0.003558,-0.008439,0.015536,0.036469,1.000000,0.013926,-0.027176,-0.000818,-0.052460,...,0.067594,-0.024618,-0.019371,-0.017857,-0.027858,0.009899,0.055699,0.013616,-0.028229,-0.003683
x7,-0.006068,0.000466,-0.025133,0.003599,-0.003334,0.013926,1.000000,-0.048201,0.009840,0.060679,...,-0.055259,-0.009836,0.007381,0.036016,-0.019978,-0.017556,-0.027312,-0.014243,-0.021657,0.410898
x8,-0.003470,0.005991,-0.083497,0.002659,-0.011862,-0.027176,-0.048201,1.000000,0.027041,-0.005817,...,0.001349,0.007827,-0.018834,0.012337,-0.082427,-0.074654,-0.033685,-0.020521,-0.009146,-0.060190
x9,0.047830,-0.024235,0.028410,0.022786,-0.061521,-0.000818,0.009840,0.027041,1.000000,0.050574,...,-0.035808,-0.026169,0.013160,0.022534,-0.007484,-0.013965,-0.011499,0.037198,-0.019396,0.006245
x10,-0.076999,0.038137,-0.002219,-0.011188,-0.011494,-0.052460,0.060679,-0.005817,0.050574,1.000000,...,-0.048710,-0.001685,-0.017533,-0.005616,0.002042,0.019123,0.012196,0.021419,0.013679,0.019843


In [108]:
X = data.drop(columns=["y"])
y = data['y']

In [109]:
# from sklearn.linear_model import LinearRegression

# def calculate_metrics(X, y):
#     model = LinearRegression()
#     model.fit(X, y)

#     y_pred = model.predict(X)

#     r_squared = model.score(X, y)

#     n = X.shape[0]
#     k = X.shape[1]
#     adjusted_r_squared = 1 - (1 - r_squared) * (n - 1) / (n - k - 1)

    
#     rmse = np.sqrt(mean_squared_error(y, y_pred))

    
#     mae = mean_absolute_error(y, y_pred)

    
#     medae = median_absolute_error(y, y_pred)

    
#     mape = np.mean(np.abs((y - y_pred) / y)) * 100


#     smape = np.mean(2 * np.abs(y - y_pred) / (np.abs(y) + np.abs(y_pred))) * 100


#     medape = np.median(np.abs((y - y_pred) / y)) * 100

#     return {
#         'RMSE': rmse,
#         'MAE': mae,
#         'MedAE': medae,
#         'R-squared': r_squared,
#         'Adjusted R-squared': adjusted_r_squared,
#         'MAPE': mape,
#         'sMAPE': smape,
#         'MedAPE': medape
#     }



 
# metrics = calculate_metrics(X, y)
# for metric_name, metric_value in metrics.items():
#     print(f"{metric_name}: {metric_value:.2f}")


In [110]:
def calculate_metrics(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    
    r_squared = model.score(X_test, y_test)
    
    n = X_test.shape[0]
    k = X_test.shape[1]
    
    adjusted_r_squared = 1 - (1 - r_squared) * (n - 1) / (n - k - 1)

    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    
    mae = mean_absolute_error(y_test, y_pred)
    
    medae = median_absolute_error(y_test, y_pred)
    
    mape = np.mean(np.abs((y_test - y_pred) / y_test))
    
    smape = np.mean(2 * np.abs(y_test - y_pred) / (np.abs(y_test) + np.abs(y_pred))) * 100
    
    medape = np.median(np.abs((y_test - y_pred) / y_test)) * 100


    return {
        'R2': r_squared,
        'Adjusted R-squared': adjusted_r_squared,
        'RMSE': rmse,
        'MAE': mae,
        'MedAE': medae,
        'MAPE': mape,
        'sMAPE': smape,
        'MedAPE': medape
    }


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

models = {
    'LinearRegression': LinearRegression(),
    'Ridge': Ridge(alpha=1),
    'Lasso': Lasso(alpha=1),
    # 'ElasticNet': ElasticNet(),
    'TheilSenRegressor': TheilSenRegressor(),
    'HuberRegressor': HuberRegressor(),
    'RANSACRegressor': RANSACRegressor()
}

predictions = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    predictions[name] = model.predict(X_test)


for model_name, model in models.items():
    print(f"Metrics for {model_name}:")
    
    metrics = calculate_metrics(model, X_train, X_test, y_train, y_test)
    
    for metric_name, metric_value in metrics.items():
        print(f"  {metric_name}: {metric_value:.2f}")
    print("\n")

Metrics for LinearRegression:
  R2: 1.00
  Adjusted R-squared: 1.00
  RMSE: 0.40
  MAE: 0.32
  MedAE: 0.27
  MAPE: 0.03
  sMAPE: 1.87
  MedAPE: 0.28


Metrics for Ridge:
  R2: 1.00
  Adjusted R-squared: 1.00
  RMSE: 0.44
  MAE: 0.36
  MedAE: 0.31
  MAPE: 0.03
  sMAPE: 1.87
  MedAPE: 0.26


Metrics for Lasso:
  R2: 1.00
  Adjusted R-squared: 1.00
  RMSE: 2.22
  MAE: 1.80
  MedAE: 1.53
  MAPE: 0.05
  sMAPE: 3.79
  MedAPE: 1.38


Metrics for TheilSenRegressor:
  R2: 1.00
  Adjusted R-squared: 1.00
  RMSE: 0.40
  MAE: 0.32
  MedAE: 0.28
  MAPE: 0.03
  sMAPE: 2.07
  MedAPE: 0.27


Metrics for HuberRegressor:
  R2: 1.00
  Adjusted R-squared: 1.00
  RMSE: 0.40
  MAE: 0.32
  MedAE: 0.27
  MAPE: 0.03
  sMAPE: 1.94
  MedAPE: 0.27


Metrics for RANSACRegressor:
  R2: 1.00
  Adjusted R-squared: 1.00
  RMSE: 0.40
  MAE: 0.32
  MedAE: 0.27
  MAPE: 0.03
  sMAPE: 1.87
  MedAPE: 0.28




In [111]:
y_pred_df = pd.DataFrame(predictions)

y_mean = y_pred_df.mean(axis=1)

y_mean


0       45.110751
1       53.306800
2      -67.066309
3     -169.485021
4       -0.992436
          ...    
195    105.556696
196    105.327785
197     67.138569
198    154.434651
199   -102.451622
Length: 200, dtype: float64

In [112]:
y_pred_df

,LinearRegression,Ridge,Lasso,TheilSenRegressor,HuberRegressor,RANSACRegressor
0,45.273190,45.186793,44.362930,45.271933,45.296469,45.273190
1,53.541258,53.462357,52.220148,53.517853,53.557928,53.541258
2,-67.290441,-67.185825,-66.146392,-67.221531,-67.263225,-67.290441
3,-169.849909,-169.640109,-167.865387,-169.867449,-169.837365,-169.849909
4,-1.288842,-1.317155,0.566181,-1.307366,-1.318591,-1.288842
...,...,...,...,...,...,...
195,105.781961,105.635101,104.589709,105.794200,105.757246,105.781961
196,105.651721,105.547623,103.760366,105.670893,105.684383,105.651721
197,67.494570,67.390469,65.429493,67.511316,67.510995,67.494570
198,154.986985,154.800855,151.836658,154.991572,155.004849,154.986985


In [113]:
def calculate_metrics(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    # y_pred = model.predict(X_test)
    
    
    r_squared = model.score(X_test, y_test)
    
    n = X_test.shape[0]
    k = X_test.shape[1]
    
    adjusted_r_squared = 1 - (1 - r_squared) * (n - 1) / (n - k - 1)

    rmse = np.sqrt(mean_squared_error(y_test, y_mean))
    
    mae = mean_absolute_error(y_test, y_mean)
    
    medae = median_absolute_error(y_test, y_mean)
    
    mape = np.mean(np.abs((y_test - y_mean) / y_test))
    
    smape = np.mean(2 * np.abs(y_test - y_mean) / (np.abs(y_test) + np.abs(y_mean))) * 100
    
    medape = np.median(np.abs((y_test - y_mean) / y_test)) * 100


    return {
        'R2': r_squared,
        'Adjusted R-squared': adjusted_r_squared,
        'RMSE': rmse,
        'MAE': mae,
        'MedAE': medae,
        'MAPE': mape,
        'sMAPE': smape,
        'MedAPE': medape
    }


# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# models = {
#     'LinearRegression': LinearRegression(),
#     'Ridge': Ridge(),
#     'Lasso': Lasso(),
#     'ElasticNet': ElasticNet(),
#     'TheilSenRegressor': TheilSenRegressor(),
#     'HuberRegressor': HuberRegressor(),
#     'RANSACRegressor': RANSACRegressor()
# }


# predictions = {}
# for name, model in models.items():
#     model.fit(X_train, y_train)
#     predictions[name] = model.predict(X_test)


for model_name, model in models.items():
    print(f"Metrics for {model_name}:")
    
    metrics = calculate_metrics(model, X_train, X_test, y_train, y_test)
    
    for metric_name, metric_value in metrics.items():
        print(f"  {metric_name}: {metric_value:.2f}")
    print("\n")

Metrics for LinearRegression:
  R2: 1.00
  Adjusted R-squared: 1.00
  RMSE: 0.55
  MAE: 0.44
  MedAE: 0.37
  MAPE: 2.95
  sMAPE: 152.08
  MedAPE: nan


Metrics for Ridge:
  R2: 1.00
  Adjusted R-squared: 1.00
  RMSE: 0.55
  MAE: 0.44
  MedAE: 0.37
  MAPE: 2.95
  sMAPE: 152.08
  MedAPE: nan


Metrics for Lasso:
  R2: 1.00
  Adjusted R-squared: 1.00
  RMSE: 0.55
  MAE: 0.44
  MedAE: 0.37
  MAPE: 2.95
  sMAPE: 152.08
  MedAPE: nan


Metrics for TheilSenRegressor:
  R2: 1.00
  Adjusted R-squared: 1.00
  RMSE: 0.55
  MAE: 0.44
  MedAE: 0.37
  MAPE: 2.95
  sMAPE: 152.08
  MedAPE: nan


Metrics for HuberRegressor:
  R2: 1.00
  Adjusted R-squared: 1.00
  RMSE: 0.55
  MAE: 0.44
  MedAE: 0.37
  MAPE: 2.95
  sMAPE: 152.08
  MedAPE: nan


Metrics for RANSACRegressor:
  R2: 1.00
  Adjusted R-squared: 1.00
  RMSE: 0.55
  MAE: 0.44
  MedAE: 0.37
  MAPE: 2.95
  sMAPE: 152.08
  MedAPE: nan




In [114]:
from sklearn.ensemble import VotingRegressor